# Предсказание отклика на рекламную рассылку
## Цель:
    Создание модели для предсказания откликак клиентов на рекламную рассылку
## Задчи:
    1. Подготовка данных
    2. Анализ данных 
    3. Создание модели по предсказанию участия в рекламной компании
    4. Создание модели для предсказания, какое из предложений примет клиент

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime

D:\anaconda\envs\datasience\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\envs\datasience\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\anaconda\envs\datasience\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


## Подготовка данных

In [2]:
data = pd.read_excel('marketing_campaign.xlsx')

In [3]:
data['ID'].fillna(0, inplace=True)

In [4]:
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


Посмотри на типы данных

In [5]:
data.dtypes

ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Z_CostContact            int64
Z_Revenue                int64
Response                 int64
dtype: object

Получим из даты рождения возраст на текущий год 

In [6]:
data['Age'] = 2021 - data['Year_Birth']
data.drop('Year_Birth', axis=1, inplace=True)

Дальше посмотрим на колонку с данными об образовании

In [7]:
data.Education.value_counts()

Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: Education, dtype: int64

Конвертиркуем строквую переменную в качественую, где цифра будем обозначать стпень образованияю. Проверим переменную на пропуски 

In [8]:
data.Education.isnull().sum()

0

In [9]:
dict_EDUC = {key:num for num, key in enumerate(data.Education.unique())}
data.Education = data.Education.map(dict_EDUC)

Проделаем тоже самое с семейным положение

In [10]:
data.Marital_Status.isnull().sum()

0

In [11]:
dict_MARIT = {key:num for num, key in enumerate(data.Marital_Status.unique())}
data.Marital_Status = data.Marital_Status.map(dict_MARIT)

С помощью колонки даты присоеденения к программе, получим количество дней которые челвек находиться в системе 

In [12]:
data['Dt_Customer'].head()

0    2012-09-04
1    2014-03-08
2    2013-08-21
3    2014-02-10
4    2014-01-19
Name: Dt_Customer, dtype: object

In [13]:
nowaday = datetime.strptime('2021-07-07', '%Y-%m-%d')

In [14]:
data['In_program'] = nowaday - pd.to_datetime(data['Dt_Customer'])
data['In_program']= data['In_program'].map(lambda x:int( x.value/86400/1000000000))

In [15]:
data.In_program.head()

0    3228
1    2678
2    2877
3    2704
4    2726
Name: In_program, dtype: int64

In [16]:
data.drop('Dt_Customer', axis=1, inplace=True)

Проверим данные на пропуски 

In [17]:
data.isnull().sum()

ID                      0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
Age                     0
In_program              0
dtype: int64

Пропуски имеет только колонка с месечным доходом. Заполним эти пропуски средним значением 

In [18]:
data['Income'].mean()

52247.25135379061

In [19]:
data.loc[data['Income'].isnull(), 'Income'] = data['Income'].mean()

Мы избавились от пропусков. Теперь создадим новую переменную, с количеством участия в акциях, которую будем использовать при анализе данных

In [20]:
data['Num_program']=0
for i in range(1,6):
    data['Num_program']+= data[f'AcceptedCmp{i}']

# Анализ данных
Дальше попробуем получить полезную информацию и описать нашего пользователя 

### Опишем семейное положение нашего пользователя используя колонки:
    Education,
    Marital,
    Teenhome,
    Kidhome,
    Income

Выведем часто встречаемые значения из колонок

In [21]:
for_pr_ED = {dict_EDUC[i]:i for i in dict_EDUC}
for_pr_MAR = {dict_MARIT[i]:i for i in dict_MARIT}
for col in ['Education','Marital_Status','Teenhome','Kidhome','Income']:
    if col=='Education':
        mod= data[col].mode()[0]
        print(col, for_pr_ED[mod])
    elif col=='Marital_Status':
        mod= data[col].mode()[0]
        print(col, for_pr_MAR[mod])
    else:
        print(col, data[col].mode()[0])

Education Graduation
Marital_Status Married
Teenhome 0
Kidhome 0
Income 52247.25135379061


И так, наш пользователь чаще всего с высшим образованием, женатый, без детей и с заработком 52247. Следующиее на что мы посмотрим это возраст наших клиентов.

In [22]:
# sns.distplot(data['Age'])
import plotly.figure_factory as ff
fig = ff.create_distplot([data['Age'].values],['Возраст'])
fig.update_layout(title='Распределение возраста')
fig.show()

По графику видно, что у нас в данных есть клиенты, чей возраст превышает 100 лет, вероятнее всего они уже клиенты небесных магазинов, удалим пользователей чей возраст больше 80.

In [23]:
data.drop(data.loc[data['Age']>80,:].index, inplace=True)

Следующее, что я хочу посмотреть, количество дней нахождения в клиентской программе.

In [24]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [25]:
fig = go.Figure()
fig.add_trace(go.Box(y=data.In_program, name='Дни'))
fig.update_layout(title='Распределение срока нахождения в базе')

График говорит, что большинство клиентов с нами уже больше 8 лет. Возможно наши данные устаревшие.

Тепеь построим график, и псмотриим количество участий в рекламных акциях

In [26]:
fig = go.Figure()
label = data.Num_program.value_counts().index
values = data.Num_program.value_counts().values
fig.add_trace(go.Pie(values=values, labels=label))
fig.update_layout(title= 'Количество участий в акциях')

Большинство наших пользователей ни разу не принимало участия в рекламных акциях. Возможно это новые пользователи, сравним их время пребывание в системе с остальными.

In [27]:
zero_parc = data.loc[data.Num_program==0, 'In_program']
other = data.loc[data.Num_program!=0, 'In_program']
fig = go.Figure()
fig.add_trace(go.Box(y=zero_parc, name='Не принимали участие'))
fig.add_trace(go.Box(y=other, name='Принимали участие'))
fig.update_layout(title='Сравнение клиентов',showlegend=False)

По графикам видно, что между клиентами нет особых различий. Чтобы точно в этом удостовериться проведём дисперсионный анализ. Нулевая гипотеза будет заключаться в том, что выборки с числом дней не различаются, то есть пользователи не являются новыми.

In [28]:
from scipy.stats import mannwhitneyu
mannwhitneyu(x=zero_parc, y=other).pvalue

0.2792043421853573

P значение больше 0.05, мы не можем отвергнуть нулевую гипотезу, так что данные не имеют значительных различий и клиенты которые ни разу не участвовали в акциях не новые. Возможно дальше нам удастся понять почему они не принимали участия в акциях.

Сейчас посмотрим на количество клиентов участвоваших в каждой акции.

In [29]:
dict_parc = {}
for num in range(1,6):
    val = data.loc[data[f'AcceptedCmp{num}']==1, :]
    dict_parc[f'AcceptedCmp{num}'] = val

In [30]:
values = [dict_parc[key].shape[0] for key in dict_parc]
label = list(dict_parc.keys())

fig = go.Figure(data=[go.Bar(x = label, y = values)])
fig.update_layout(title='Количество участников в акциях')

Из графика можно узнать, что количество клиентов в каждой акции фактически идентичное, кроме 2. Возможно в ней радикально отличалось аукционное предложение. Посмотрим, как изменилось количество клиентов после проведения перовой акции. Так же необходимо посмотреть на пересечение клиентов, участвовавших в акциях.

In [31]:
min_day = data.loc[data.AcceptedCmp1==1, 'In_program'].min()
data.loc[(data.AcceptedCmp1!=1)&(data.In_program<min_day), :].shape[0]

2

In [32]:
keys = list(dict_parc.keys())
our_dict = {}
for i in range(len(keys)):
    df = dict_parc[keys[i]]
    for j in range(i+1, len(keys)):
        df_m = dict_parc[keys[j]]
        our = df.merge(df_m, on='ID').shape[0]
        our_dict[f"AcceptedCmp{i+1}_AcceptedCmp{j+1}"] = our

gist = [go.Bar(x=[x for x in our_dict], 
               y = [y/(dict_parc[key.split('_')[0]].shape[0]+
                       dict_parc[key.split('_')[1]].shape[0]) 
               for y,key in zip(our_dict.values(), our_dict)])]

layout = go.Layout(title='Количество общих участниоков в акциях')
go.Figure(data=gist, layout = layout)

Число общих участников акций не превышает 22% в самом большом случае и, после проведения первой акции, количество клиентов не увеличилось. Можно сделать вывод, что в акциях участвуют различные группы клиентов или разным клиентам дели разные аукционные предложения. Найду типовой протрет участника определённой акции. Это может пригодиться в дальнейшем для составления рекламных предложений. Сделаю я это с помощью метода Kmeans.

In [33]:
from sklearn.cluster import KMeans

In [34]:
id_users  = {}
for key in dict_parc:
    df = dict_parc[key]
    kmean= KMeans(n_clusters=1, n_jobs=-1)
    kmean.fit(df)
    point = kmean.cluster_centers_[0]
    distance = {np.linalg.norm(point-df.iloc[i, :].values) :i  for i in range(df.shape[0])}
    id_users[key] = distance[min(distance.keys())]

D:\anaconda\envs\datasience\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning:

'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).

D:\anaconda\envs\datasience\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

D:\anaconda\envs\datasience\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning:

'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).

D:\anaconda\envs\datasience\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

D:\anaconda\envs\datasience\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWa

In [35]:
list_df = [dict_parc[key].iloc[val, :]
            for key, val in zip(id_users.keys(), id_users.values())]

In [36]:
res_df = pd.concat(list_df, axis=1).T
res_df.index = dict_parc.keys()
res_df

,ID,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age,In_program,Num_program
AcceptedCmp1,4646.0,4.0,2.0,78497.0,0.0,0.0,44.0,207.0,26.0,447.0,...,0.0,1.0,0.0,0.0,3.0,11.0,0.0,70.0,2775.0,1.0
AcceptedCmp2,3009.0,1.0,4.0,71670.0,0.0,0.0,8.0,1462.0,16.0,128.0,...,1.0,0.0,1.0,0.0,3.0,11.0,1.0,59.0,2920.0,3.0
AcceptedCmp3,4656.0,4.0,0.0,51250.0,1.0,0.0,28.0,342.0,32.0,230.0,...,0.0,0.0,0.0,0.0,3.0,11.0,0.0,31.0,3021.0,1.0
AcceptedCmp4,5544.0,0.0,0.0,67384.0,0.0,1.0,32.0,957.0,40.0,175.0,...,0.0,0.0,0.0,0.0,3.0,11.0,0.0,51.0,3161.0,1.0
AcceptedCmp5,5341.0,4.0,3.0,81975.0,0.0,1.0,2.0,983.0,76.0,184.0,...,1.0,0.0,0.0,0.0,3.0,11.0,0.0,59.0,3105.0,1.0


Полученные данные можно учитывать в дальнейшем, для составления акционных предложений. Сохраню их в csv файл, для удобства использования их другими членами команды.

In [37]:
res_df.to_csv('Tipical_user.csv')

###  Мы нашли типового пользователя, и выяснили что в каждой акции участвуют разные группы пользователей. Сейчас проанализируем информацию о средних тратах на определённую продукцию. Данные буду получены из колонок:
    MntFishProducts
    MntMeatProducts
    MntFruits
    MntSweetProducts 
    MntWines
    MntGoldProds 

In [38]:
contrast_col = ['MntFishProducts',
'MntMeatProducts',
'MntFruits',
'MntSweetProducts', 
'MntWines',
'MntGoldProds']

In [39]:
dict_parc['Didn\'t parc'] = data.loc[data.Num_program==0,:]

name_x= contrast_col
gist_value = [[df.loc[:, col].mean() for col in contrast_col] for df in dict_parc.values()]
gist_list_2 = [go.Bar(x=name_x, 
                    y =val,
                    name=name) 
             for name, val in zip(dict_parc, gist_value)]
layout = go.Layout(barmode='group', title='Средня сумма в зависмости от категории')
go.Figure(data=gist_list_2, layout= layout)

По графику видно, что у клиентов, принимавших участие в акциях средний чек фактически по всем позициям выше, чем у пассивных клиентов. И особенно чек выше на по позициям вино и мясные продукты. Видимо исходя из среднего чека клиентам и делали акционные предложения. Возможно нам и не нужна модель чтобы предсказать участие клиента в акции, а достаточно делать акционное предложения исходя из среднего чека вина и мясных продуктов.

Так же есть вероятность, что клиенты не был проинформирован об акции или не пeрешёл по ссылке из электронного письма. Посмотрим на среднее посещения сайта клиентом в зависимости от числа принятия участий в акциях.

In [40]:
fig = go.Figure()
index = data.Num_program.value_counts().index
values = [data.loc[data.Num_program==i, 'NumWebVisitsMonth'].mean()  for i in index]
fig.add_trace(go.Pie(values=values, labels=index))
fig.update_layout(title='Среднее количество посежений сайта в зависиимости от кол-во участия')
fig.show()

Результат очень интересный. Люди, которые не участвовали в акциях, в среднем больше заходят на сайт, то есть они были проинформированы об акциях. Возможно им просто не понравилось предложение.
Есть ещё одна деталь которую я возможно упустил. Клиенты, не участвующие в акциях это есть тестовая группа для предсказания подходящей акции, но в задании об этом ничего не сказано, так что буду считать их неотзывчивыми клиентами.

In [41]:
data.NumWebVisitsMonth.mean()

5.319320214669052

Среднее количество посещения сайта в месяц составляет 5 среди всех клиентов, посмотрим теперь среднее число покупок онлайн и офлайн

In [42]:
name_x= ['Онлайн', 'Офлайн']
gist_list_in = [go.Bar(x=name_x, 
                    y = [dict_parc[key][col].mean() for col in ['NumWebPurchases', 'NumStorePurchases']],
                    name=key) for key in dict_parc]

layout = go.Layout(barmode='group', title='Сравнение продаж онлайн и офлайн')
go.Figure(data=gist_list_in, layout= layout)

По графикам мы видим, что акции стимулирует онлайн продажи, хотя участники акции в среднем на портал заходят не чаще пассивных клиентов, что следовало из предыдущего графика.

# Сделаем выводы из иследования:
1. 79% процентов наших клиентов не участвуют в акциях, и это не новые клиенты
2. В каждой акции участвовала определённая группа людей, пересечение групп не превышает 22%, и некоторые клиенты принимали участие в нескольких акциях. Так же мы нашли типовой профиль клиента для каждой акции. 
3. Средний чек у участников акции значительно выше по всем позициям, чем у пассивных клиентов. Самые больший продажи составляют вино и мясные продукты.
4. Пассивные клиенты в среднем чаще заходят на сайт компании, то есть они были проинформированы о акциях, но не приняли в них участия.
5. Основные продажи приходятся на офлайн. Что ещё интересно, что пассивные клиенты хоть и чаще заходят на сайт, но число покупок онлайн меньше. Возможно акциями компания стимулирует онлайн продажи.

# Построение предсказательной модели
У нас 79% клиентов ни разу не принимали участие в акциях, первая задача которую на нужно решить это будет ли клиент участвовать в какой-либо акции. 0 - не участвует, 1- участвует.

Создадим переменную Y она будет нашим таргетом, Х- данные пользователях, без колонок, содержащих информацию о участии в акции

In [43]:
Y = data.Num_program.map(lambda x: 0 if x==0 else 1)
drop_col = [f'AcceptedCmp{i}' for i in range(1,6)] + ['Num_program', 'ID', 'Response']
X = data.loc[:, [i for i in data.columns if i not in drop_col]]

In [44]:
fig = go.Figure()
fig.add_trace(go.Bar(x=[0,1], y =Y.value_counts().values))
fig.update_layout(title='Распределения классов')

У нас в данных присутствует большой дисбаланс классов, утём это при разбиении данных и обучении модели

Дальше разделим данные тест и трайн, и стандартизируем 

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1, stratify=Y)

In [47]:
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

Сразу проведём отбор модели

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [49]:
random_forest = RandomForestClassifier()
sgd = SGDClassifier()
log_reg = LogisticRegression()
des_tree = DecisionTreeClassifier()
svc = SVC()
models = [ random_forest, sgd, log_reg, des_tree, svc]

In [50]:
train_model ={mod.__str__(): mod.fit(X_train, y_train) for mod in models}

Для оценки на данном этапе буду использовать precision_score, так как ключеваю наша задача предсказание положительных объектов класса

In [51]:
from sklearn.metrics import precision_score

In [52]:
for mod in train_model:
    pred =train_model[mod].predict(X_test)
    print(mod, precision_score(y_test, pred))

RandomForestClassifier() 0.9324324324324325
SGDClassifier() 0.5544554455445545
LogisticRegression() 0.7466666666666667
DecisionTreeClassifier() 0.47435897435897434
SVC() 0.7777777777777778


По полученным результатам для дальнейшего подбора гиперпараметров возьмём модель RandomForestClassifier

Первый параметр которые я очу настроить это веса классов.

In [53]:
weight = [{1:i, 0:1-i} for i in np.arange(0.1, 1, 0.1)]
res={}
for i in weight:
    forest = RandomForestClassifier(class_weight=i, n_jobs=-1)
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    key = precision_score(y_test, y_pred)
    res[key] = forest

In [54]:
max(res.keys())

0.9298245614035088

Мы доброди веса для классов и увеличили оценку нашей модели, сейчас будем подбирать остальные параметры

In [55]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [100,200,300,400,500,600,700,600]
max_features = ['sqrt']
max_depth = [11,15,20, 25, 30]
min_samples_split = [2,3,4,22,23,24]
min_samples_leaf = [2,3,4,5,6,7]
bootstrap = [False]
weight = [{1:i, 0:1-i} for i in np.arange(0.1, 1, 0.1)]

param_grid_forest = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': weight}
new_param={}
for i in range(10):
    new_forest =RandomizedSearchCV(random_forest, param_grid_forest,n_iter=10, cv =3, verbose =1, n_jobs=-1, scoring='precision')
    new_forest.fit(X_train, y_train)
    pred= new_forest.predict(X_test)
    key = precision_score(y_test, pred)
    new_param[key] = new_forest

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [56]:
max(new_param.keys())

1.0

Лучшая оценка модели после подбора параметров составляет 1. Ещё попробуем библиотеку flaml для поиска модели

In [57]:
from flaml import AutoML

Нам ещё понадобиться валидационная выборка

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

In [59]:
auto_ml= AutoML()
auto_ml.fit(X_train, y_train, X_val, y_val, metric = 'roc_auc', task='classification', n_splits=3)

[flaml.automl: 07-13 23:12:13] {893} INFO - Evaluation method: cv
[flaml.automl: 07-13 23:12:13] {596} INFO - Using StratifiedKFold
[flaml.automl: 07-13 23:12:13] {914} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 07-13 23:12:13] {934} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 07-13 23:12:13] {998} INFO - iteration 0, current learner lgbm
[flaml.automl: 07-13 23:12:13] {1150} INFO -  at 0.1s,	best lgbm's error=0.2375,	best lgbm's error=0.2375
[flaml.automl: 07-13 23:12:13] {998} INFO - iteration 1, current learner lgbm
[flaml.automl: 07-13 23:12:13] {1150} INFO -  at 0.2s,	best lgbm's error=0.2375,	best lgbm's error=0.2375
[flaml.automl: 07-13 23:12:13] {998} INFO - iteration 2, current learner lgbm
[flaml.automl: 07-13 23:12:13] {1150} INFO -  at 0.3s,	best lgbm's error=0.2162,	best lgbm's error=0.2162
[flaml.automl: 07-13 23:12:13] {998} INFO - iteration 3, current learner lgbm
[flaml.automl: 07-13

In [60]:
y_auto = auto_ml.predict(X_test)
precision_score(y_test, y_auto)

0.7804878048780488

In [61]:
from sklearn.metrics import roc_auc_score, precision_score

In [62]:
for mod in [auto_ml,new_param[max(new_param.keys())], res[max(res.keys())], train_model['RandomForestClassifier()']]:
    pred = mod.predict(X_test)
    print("MODEL", mod.__str__(), 'SCORE',precision_score(y_test, pred),"*"*20, sep='\n')

MODEL
SCORE
0.7804878048780488
********************
MODEL
RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [False],
                                        'class_weight': [{0: 0.9, 1: 0.1},
                                                         {0: 0.8, 1: 0.2},
                                                         {0: 0.7,
                                                          1: 0.30000000000000004},
                                                         {0: 0.6, 1: 0.4},
                                                         {0: 0.5, 1: 0.5},
                                                         {0: 0.4, 1: 0.6},
                                                         {0: 0.29999999999999993,
                                                          1: 0.7000000000000001},
                                                         {0: 0.19999999999999996,
                                      

Сейчас построим график feature_importances

In [63]:
best_mod = new_param[max(new_param.keys())]
values = best_mod.best_estimator_.feature_importances_
labels = list(X.columns)
fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y =values))
fig.update_layout(title='Feature importances')

По графику видно что основной вклад как и предполагалось внесли колонки MntWines, MntMeatProducts n Income. Как я и упоминал выше, для определения, будет ли участвовать клиент в акции, не обязательно обучать модель, а достаточно установить пороговые значения для колонок.

И так лучшую метрику показывает случайный лес. Результат отличный мы со 100% точностью предсказываем положительный класс(так мы подбирали параметры с помощью RandomizedSearchCV при повторном запуске мы можем не получить такой результат, сохраню модель в файл). Следующая задача, которую предстоит решить- это предсказание какое предложение примет клиент. Будем предсказывать бинарный вектор с помощью линейной регрессии, где индекс 1 означает номер предложения которое примет клиент

In [64]:
# import pickle

# with open('Best_forest_model_2.pkl', 'wb') as f:
#     pickle.dump(new_param[max(new_param.keys())].best_estimator_, f)

In [65]:
Full_df = data.loc[data.Num_program!=0, :]

Я создал датафрейм для предсказания. Сейчас разделим его на трейн и тест и стандартизируем и обучим модель линейной регресси. 

In [66]:
Y_mcl = Full_df.loc[:, [f"AcceptedCmp{i}" for i in range(1,6)]]
X_mcl = Full_df.loc[:, [i for i in df.columns if i not in drop_col+['class']]]

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X_mcl, Y_mcl, test_size=0.3, random_state=31)

In [68]:
# std_cl = StandardScaler()
X_train = std.transform(X_train)
X_test = std.transform(X_test)

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

Обучил модель, теперь посмотрим какие значения она выдаёт в качестве предсказания.

In [71]:
pred= lin_reg.predict(X_test)
pred[0]

array([-0.08234011,  0.11845274,  0.78594625,  0.27664484, -0.20153692])

Не очень похоже на бинарный вектор) Максимальный элемент в этом векторе будем считать за единицу, а его индекс- это номер акции в которой пользователь скорее всего примет участие. Остальные значения можно интерпретировать как вероятность участи в других акциях. Как мы помним клиент может принять участие сразу в нескольких акциях, но я предсказываю участие в только одной акции, я считаю этот результат приемлемым, так мы предсказали акции в которой клиент примет участие с наибольшей вероятность.

Как мы измерим качество предсказаний? Создадим свою метрику. Вектор предсказаний- вектор чисел от 0 до 1, а вектор для сравнения состоит из 0 и 1 и может содержать несколько 1. Наша метрика возвращает 1, если индекс максимального числа вектора предсказаний совпадает хотя бы с одним индексом 1 в векторе сравнения.

In [72]:
def new_metrics(y_pred, y_true):
    arg_max = y_pred.argmax()
    one_ind = np.where(y_true==1)
    return 1 if arg_max in one_ind[0] else 0

In [73]:
score = [new_metrics(y_pred, y_true) for y_pred, y_true in zip(pred, y_test.values)]

In [74]:
sum(score)/len(score)

0.7266187050359713

In [75]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=3,specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [ {"type": "bar"},  {"type": "bar"}, None]], shared_xaxes=True,shared_yaxes=True)
col =1
row=1
labels = list(X.columns)
coef = lin_reg.coef_
for i in range(coef.shape[0]):
    if col==4:
        row+=1
        col=1
    fig.add_trace(go.Bar(x=labels,y=coef[i], name=f"AcceptedCmp{i}"), row=row, col=col)
    col+=1
fig.update_layout(title='Feature importances for LinearRegression')
fig.show()

Мы построили график feature importances для линейной регрессии. На нём видно какой вклад вносила каждая фича для предсказания определённого числа в векторе. Для каждого числа коэффициента разные, но, как и в предыдущей модели, весомы вклад вносит колонка Incoms

Точность модели составляет 72%, считаю, что это хороший результат. Так же значения вероятностей участия в других акциях из вектора предсказаний можно использовать для формирования индивидуального предложения для клиента комбинирую условия акций, которые имеют высокую вероятность.

# Подведейм итоги по обучению:
1. Мне удалось обучить модель предсказнию участия в акции с оценкой равной 1.0. Это отличный резульатат. Возможно я где то ошибся, в данный момент я не вижу этой ошибки, если вы её увидите прошу насать мне.
2. Задачу по предсказанию какое предложение примет клиент решил с помощью линейоной регрусси, точность составляет 72%. В случаи если этот результат или  метод решения клиента не устроит, прошу написать мне, будем что то менять.
# Вывод
В ходу работы на этим проектом удалось обучить модель предсказания. Осталось тольк протестить её в боевых условиях